In [1]:
import os
os.chdir('..') # this resolves ImportError: attempted relative import with no known parent package
import pandas as pd
import emoji
from bs4 import BeautifulSoup 
import html
import contractions
import re
pd.set_option('display.max_colwidth', None)

In [2]:
# import data from csv
raw_data = pd.read_csv("data/raw/new_character_reveal_comments.csv")
data = raw_data.copy()
df = pd.DataFrame(data)
df.head()

,videoId,authorDisplayName,publishedAt,updatedAt,likeCount,totalReplyCount,textDisplay
0,rDxrpSqYHD8,@silveriver9,2023-11-01 16:09:58+00:00,2023-11-01 16:10:43+00:00,4,4,First. Now where is LEI WULONG?!
1,rDxrpSqYHD8,@faizaanjaved7150,2023-11-01 16:10:05+00:00,2023-11-01 16:10:05+00:00,1,1,Already seen it. Ur getting less view&#39;s now bamco
2,rDxrpSqYHD8,@TS-rw4lk,2023-11-01 16:10:05+00:00,2023-11-01 16:10:05+00:00,0,0,wow
3,rDxrpSqYHD8,@ALONCAK,2023-11-01 16:10:06+00:00,2023-11-01 16:10:06+00:00,0,0,Oww yeaah
4,rDxrpSqYHD8,@Rough_Estimates,2023-11-01 16:10:06+00:00,2023-11-01 16:10:06+00:00,135,14,I hope we get an angel version of Jin


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2041 entries, 0 to 2040
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   videoId            2041 non-null   object
 1   authorDisplayName  2041 non-null   object
 2   publishedAt        2041 non-null   object
 3   updatedAt          2041 non-null   object
 4   likeCount          2041 non-null   int64 
 5   totalReplyCount    2041 non-null   int64 
 6   textDisplay        2041 non-null   object
dtypes: int64(2), object(5)
memory usage: 111.7+ KB


In [4]:
df.videoId.nunique()

1

In [5]:
# remove emojis, backslashes, stop words etc.

In [6]:
# investigate comments
df[['videoId', 'publishedAt', 'textDisplay']].tail(20)

,videoId,publishedAt,textDisplay
2021,rDxrpSqYHD8,2023-12-12 23:28:13+00:00,I got jin kazama tatooo❤
2022,rDxrpSqYHD8,2023-12-13 14:51:33+00:00,Armor king????????
2023,rDxrpSqYHD8,2023-12-13 19:42:25+00:00,bring back eddy
2024,rDxrpSqYHD8,2023-12-13 20:03:01+00:00,Great roster. Can we see Bruce Irvin in the future? I miss that kickboxing style
2025,rDxrpSqYHD8,2023-12-14 01:14:17+00:00,Alisa my love! Such as Kula too
2026,rDxrpSqYHD8,2023-12-14 01:16:08+00:00,Is that jims mother in the roaster im excited to play with her
2027,rDxrpSqYHD8,2023-12-15 06:09:22+00:00,how panda gets a spot is beyond me just make it a reskin of kuma ffs
2028,rDxrpSqYHD8,2023-12-15 12:57:04+00:00,32 Characters off the bat? LETTTTTTTTTTTTTTTS GO
2029,rDxrpSqYHD8,2023-12-15 14:53:14+00:00,no eddy :(
2030,rDxrpSqYHD8,2023-12-17 05:44:57+00:00,"NEW ROSTER LOOKS SO GOOD!!! But where is the big man himself, BOB?!"


# Cleaning tasks

There are many steps needded to process and clean the text:


<input type="checkbox" checked> removing emojis

<input type="checkbox" checked> removing html

<input type="checkbox" checked > removing special characters and excessive punctuation (e.g., Yes!!!!!!!!!!!)

<input type="checkbox" checked> lower casing

<input type="checkbox" checked> contractions (changing "don't" to "do not")


# Removi emojis

In [7]:
def remove_emojis(text):
    """Removes emojis from the input text.

    Parameters
    -----------
    text : str
        A string of text.

    Returns
    --------
    text : str
        The string of text without emojis.

    """
    return emoji.replace_emoji(text, "")

In [8]:
df_emojiless = df.copy()
df_emojiless['textDisplay'] = df_emojiless['textDisplay'].apply(remove_emojis)
df_emojiless.tail(20)

,videoId,authorDisplayName,publishedAt,updatedAt,likeCount,totalReplyCount,textDisplay
2021,rDxrpSqYHD8,@GEObandicoot,2023-12-12 23:28:13+00:00,2023-12-12 23:28:13+00:00,0,0,I got jin kazama tatooo
2022,rDxrpSqYHD8,@jimmysaints7894,2023-12-13 14:51:33+00:00,2023-12-13 14:51:33+00:00,0,0,Armor king????????
2023,rDxrpSqYHD8,@im_nbc2548,2023-12-13 19:42:25+00:00,2023-12-13 19:42:25+00:00,1,0,bring back eddy
2024,rDxrpSqYHD8,@mecrac,2023-12-13 20:03:01+00:00,2023-12-13 20:03:01+00:00,0,0,Great roster. Can we see Bruce Irvin in the future? I miss that kickboxing style
2025,rDxrpSqYHD8,@Marucho-qf2cr,2023-12-14 01:14:17+00:00,2023-12-14 01:14:17+00:00,0,0,Alisa my love! Such as Kula too
2026,rDxrpSqYHD8,@kelvinz29,2023-12-14 01:16:08+00:00,2023-12-14 01:16:08+00:00,0,0,Is that jims mother in the roaster im excited to play with her
2027,rDxrpSqYHD8,@johnjohnson4834,2023-12-15 06:09:22+00:00,2023-12-15 06:09:22+00:00,1,0,how panda gets a spot is beyond me just make it a reskin of kuma ffs
2028,rDxrpSqYHD8,@noyce.,2023-12-15 12:57:04+00:00,2023-12-15 12:57:04+00:00,1,0,32 Characters off the bat? LETTTTTTTTTTTTTTTS GO
2029,rDxrpSqYHD8,@ssj3134,2023-12-15 14:53:14+00:00,2023-12-15 14:53:14+00:00,1,0,no eddy :(
2030,rDxrpSqYHD8,@andreypastrana810,2023-12-17 05:44:57+00:00,2023-12-17 05:44:57+00:00,0,0,"NEW ROSTER LOOKS SO GOOD!!! But where is the big man himself, BOB?!"


# Lower case (normalise) the text

In [9]:
def normalize_text(text):
    """Lower cases the input text.

    Parameters
    -----------
    text : str
        A string of text.

    Returns
    --------
    text : str
        The string of text all in lower case.

    """
    return text.lower()

In [10]:
df_lower_cased = df.copy()
df_lower_cased['textDisplay'] = df['textDisplay'].apply(normalize_text)
df_lower_cased.tail(20)

,videoId,authorDisplayName,publishedAt,updatedAt,likeCount,totalReplyCount,textDisplay
2021,rDxrpSqYHD8,@GEObandicoot,2023-12-12 23:28:13+00:00,2023-12-12 23:28:13+00:00,0,0,i got jin kazama tatooo❤
2022,rDxrpSqYHD8,@jimmysaints7894,2023-12-13 14:51:33+00:00,2023-12-13 14:51:33+00:00,0,0,armor king????????
2023,rDxrpSqYHD8,@im_nbc2548,2023-12-13 19:42:25+00:00,2023-12-13 19:42:25+00:00,1,0,bring back eddy
2024,rDxrpSqYHD8,@mecrac,2023-12-13 20:03:01+00:00,2023-12-13 20:03:01+00:00,0,0,great roster. can we see bruce irvin in the future? i miss that kickboxing style
2025,rDxrpSqYHD8,@Marucho-qf2cr,2023-12-14 01:14:17+00:00,2023-12-14 01:14:17+00:00,0,0,alisa my love! such as kula too
2026,rDxrpSqYHD8,@kelvinz29,2023-12-14 01:16:08+00:00,2023-12-14 01:16:08+00:00,0,0,is that jims mother in the roaster im excited to play with her
2027,rDxrpSqYHD8,@johnjohnson4834,2023-12-15 06:09:22+00:00,2023-12-15 06:09:22+00:00,1,0,how panda gets a spot is beyond me just make it a reskin of kuma ffs
2028,rDxrpSqYHD8,@noyce.,2023-12-15 12:57:04+00:00,2023-12-15 12:57:04+00:00,1,0,32 characters off the bat? lettttttttttttttts go
2029,rDxrpSqYHD8,@ssj3134,2023-12-15 14:53:14+00:00,2023-12-15 14:53:14+00:00,1,0,no eddy :(
2030,rDxrpSqYHD8,@andreypastrana810,2023-12-17 05:44:57+00:00,2023-12-17 05:44:57+00:00,0,0,"new roster looks so good!!! but where is the big man himself, bob?!"


# Remove html

In [11]:
# this function returned a Warning due to some (most) comments not containing any html:
# Warning: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
# so, although the warning was harmless and the function worked, it is preferable to use html.unescape(text) to handle the 'html entities' such as "&quot;"

def remove_html(text):
    """Removes html entities from the input text.

    Parameters
    -----------
    text : str
        A string of text.

    Returns
    --------
    text : str
        The string of text without html.

    Notes
    ------
    Warning: MarkupResemblesLocatorWarning:
        This function returned a Warning due to some (most) comments not containing any html:
        "Warning: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and p
        ass the filehandle into Beautiful Soup."
        So, although the warning was harmless and the function worked, it is preferable to use `html.unescape(text)` to handle the 
        'html entities' such as "&quot;"

    """
    try:
        soup = BeautifulSoup(text, 'html.parser')
        return soup.get_text()

    except MarkupResemblesLocatorWarning:
        return text


# use html unescape to achieve the same as the above
def remove_html_unescape(text):
    """Removes html entities from the input text.

    Parameters
    -----------
    text : str
        A string of text.

    Returns
    --------
    text : str
        The string of text without html.

    """
    return html.unescape(text)

In [12]:
sentence = "&quot;marvelous!&quot; 😂"
remove_html(sentence)

'"marvelous!" 😂'

In [13]:
remove_html_unescape(sentence)

'"marvelous!" 😂'

In [14]:
df_no_html = df.copy()
df_no_html["textDisplay"] = df_no_html["textDisplay"].apply(remove_html_unescape)
df_no_html.tail(20)

,videoId,authorDisplayName,publishedAt,updatedAt,likeCount,totalReplyCount,textDisplay
2021,rDxrpSqYHD8,@GEObandicoot,2023-12-12 23:28:13+00:00,2023-12-12 23:28:13+00:00,0,0,I got jin kazama tatooo❤
2022,rDxrpSqYHD8,@jimmysaints7894,2023-12-13 14:51:33+00:00,2023-12-13 14:51:33+00:00,0,0,Armor king????????
2023,rDxrpSqYHD8,@im_nbc2548,2023-12-13 19:42:25+00:00,2023-12-13 19:42:25+00:00,1,0,bring back eddy
2024,rDxrpSqYHD8,@mecrac,2023-12-13 20:03:01+00:00,2023-12-13 20:03:01+00:00,0,0,Great roster. Can we see Bruce Irvin in the future? I miss that kickboxing style
2025,rDxrpSqYHD8,@Marucho-qf2cr,2023-12-14 01:14:17+00:00,2023-12-14 01:14:17+00:00,0,0,Alisa my love! Such as Kula too
2026,rDxrpSqYHD8,@kelvinz29,2023-12-14 01:16:08+00:00,2023-12-14 01:16:08+00:00,0,0,Is that jims mother in the roaster im excited to play with her
2027,rDxrpSqYHD8,@johnjohnson4834,2023-12-15 06:09:22+00:00,2023-12-15 06:09:22+00:00,1,0,how panda gets a spot is beyond me just make it a reskin of kuma ffs
2028,rDxrpSqYHD8,@noyce.,2023-12-15 12:57:04+00:00,2023-12-15 12:57:04+00:00,1,0,32 Characters off the bat? LETTTTTTTTTTTTTTTS GO
2029,rDxrpSqYHD8,@ssj3134,2023-12-15 14:53:14+00:00,2023-12-15 14:53:14+00:00,1,0,no eddy :(
2030,rDxrpSqYHD8,@andreypastrana810,2023-12-17 05:44:57+00:00,2023-12-17 05:44:57+00:00,0,0,"NEW ROSTER LOOKS SO GOOD!!! But where is the big man himself, BOB?!"


In [15]:
# creates a WARNING
df_no_html = df.copy()
df_no_html["textDisplay"] = df_no_html["textDisplay"].apply(remove_html)
df_no_html.tail(20)

/var/folders/lp/914jvm397pvf1h_lk1cb1gj80000gn/T/ipykernel_10110/519258905.py:29: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


,videoId,authorDisplayName,publishedAt,updatedAt,likeCount,totalReplyCount,textDisplay
2021,rDxrpSqYHD8,@GEObandicoot,2023-12-12 23:28:13+00:00,2023-12-12 23:28:13+00:00,0,0,I got jin kazama tatooo❤
2022,rDxrpSqYHD8,@jimmysaints7894,2023-12-13 14:51:33+00:00,2023-12-13 14:51:33+00:00,0,0,Armor king????????
2023,rDxrpSqYHD8,@im_nbc2548,2023-12-13 19:42:25+00:00,2023-12-13 19:42:25+00:00,1,0,bring back eddy
2024,rDxrpSqYHD8,@mecrac,2023-12-13 20:03:01+00:00,2023-12-13 20:03:01+00:00,0,0,Great roster. Can we see Bruce Irvin in the future? I miss that kickboxing style
2025,rDxrpSqYHD8,@Marucho-qf2cr,2023-12-14 01:14:17+00:00,2023-12-14 01:14:17+00:00,0,0,Alisa my love! Such as Kula too
2026,rDxrpSqYHD8,@kelvinz29,2023-12-14 01:16:08+00:00,2023-12-14 01:16:08+00:00,0,0,Is that jims mother in the roaster im excited to play with her
2027,rDxrpSqYHD8,@johnjohnson4834,2023-12-15 06:09:22+00:00,2023-12-15 06:09:22+00:00,1,0,how panda gets a spot is beyond me just make it a reskin of kuma ffs
2028,rDxrpSqYHD8,@noyce.,2023-12-15 12:57:04+00:00,2023-12-15 12:57:04+00:00,1,0,32 Characters off the bat? LETTTTTTTTTTTTTTTS GO
2029,rDxrpSqYHD8,@ssj3134,2023-12-15 14:53:14+00:00,2023-12-15 14:53:14+00:00,1,0,no eddy :(
2030,rDxrpSqYHD8,@andreypastrana810,2023-12-17 05:44:57+00:00,2023-12-17 05:44:57+00:00,0,0,"NEW ROSTER LOOKS SO GOOD!!! But where is the big man himself, BOB?!"


# Contractions
NOTE: Contractions need to be removed after html is removed to avoid errors in the output.

In [16]:
sentence = "I'd like to, wouldn't you? It'd be great if you weren't away. What do y'all think?"
contractions.fix(sentence)

'I would like to, would not you? It would be great if you were not away. What do you all think?'

In [17]:
def process_contractions(text):
    """Expands contractions within the inut text e.g., turns "isn't" into "is not".

    Parameters
    -----------
    text : str
        A string of text.

    Returns
    --------
    text : str
        The string of text all in lower case.

    """
    return contractions.fix(text)

In [18]:
df_contractions = df.copy()
# remove html before processing contractions
df_contractions['textDisplay'] = df_contractions['textDisplay'].apply(remove_html_unescape)
df_contractions['textDisplay'] = df_contractions['textDisplay'].apply(process_contractions)
df_contractions.tail(20)

,videoId,authorDisplayName,publishedAt,updatedAt,likeCount,totalReplyCount,textDisplay
2021,rDxrpSqYHD8,@GEObandicoot,2023-12-12 23:28:13+00:00,2023-12-12 23:28:13+00:00,0,0,I got jin kazama tatooo❤
2022,rDxrpSqYHD8,@jimmysaints7894,2023-12-13 14:51:33+00:00,2023-12-13 14:51:33+00:00,0,0,Armor king????????
2023,rDxrpSqYHD8,@im_nbc2548,2023-12-13 19:42:25+00:00,2023-12-13 19:42:25+00:00,1,0,bring back eddy
2024,rDxrpSqYHD8,@mecrac,2023-12-13 20:03:01+00:00,2023-12-13 20:03:01+00:00,0,0,Great roster. Can we see Bruce Irvin in the future? I miss that kickboxing style
2025,rDxrpSqYHD8,@Marucho-qf2cr,2023-12-14 01:14:17+00:00,2023-12-14 01:14:17+00:00,0,0,Alisa my love! Such as Kula too
2026,rDxrpSqYHD8,@kelvinz29,2023-12-14 01:16:08+00:00,2023-12-14 01:16:08+00:00,0,0,Is that jims mother in the roaster i am excited to play with her
2027,rDxrpSqYHD8,@johnjohnson4834,2023-12-15 06:09:22+00:00,2023-12-15 06:09:22+00:00,1,0,how panda gets a spot is beyond me just make it a reskin of kuma ffs
2028,rDxrpSqYHD8,@noyce.,2023-12-15 12:57:04+00:00,2023-12-15 12:57:04+00:00,1,0,32 Characters off the bat? LETTTTTTTTTTTTTTTS GO
2029,rDxrpSqYHD8,@ssj3134,2023-12-15 14:53:14+00:00,2023-12-15 14:53:14+00:00,1,0,no eddy :(
2030,rDxrpSqYHD8,@andreypastrana810,2023-12-17 05:44:57+00:00,2023-12-17 05:44:57+00:00,0,0,"NEW ROSTER LOOKS SO GOOD!!! But where is the big man himself, BOB?!"


# Special characters and punctuation/excessive punctuation

In [19]:
def remove_all_punctuation(text):
    """Removes punctuation (including special characters such as "$%^&*" etc.)

    Parameters
    -----------
    text : str
        A string of text.

    Returns
    --------
    text : str
        The string of text without punctuation and special characters.

    """
    pattern = r"[^\w\s]"
    return re.sub(pattern, "", text)

In [20]:
df_punctuation_removed = df.copy()
df_punctuation_removed['textDisplay'] = df_punctuation_removed['textDisplay'].apply(remove_all_punctuation)
df_punctuation_removed.tail(20)

,videoId,authorDisplayName,publishedAt,updatedAt,likeCount,totalReplyCount,textDisplay
2021,rDxrpSqYHD8,@GEObandicoot,2023-12-12 23:28:13+00:00,2023-12-12 23:28:13+00:00,0,0,I got jin kazama tatooo
2022,rDxrpSqYHD8,@jimmysaints7894,2023-12-13 14:51:33+00:00,2023-12-13 14:51:33+00:00,0,0,Armor king
2023,rDxrpSqYHD8,@im_nbc2548,2023-12-13 19:42:25+00:00,2023-12-13 19:42:25+00:00,1,0,bring back eddy
2024,rDxrpSqYHD8,@mecrac,2023-12-13 20:03:01+00:00,2023-12-13 20:03:01+00:00,0,0,Great roster Can we see Bruce Irvin in the future I miss that kickboxing style
2025,rDxrpSqYHD8,@Marucho-qf2cr,2023-12-14 01:14:17+00:00,2023-12-14 01:14:17+00:00,0,0,Alisa my love Such as Kula too
2026,rDxrpSqYHD8,@kelvinz29,2023-12-14 01:16:08+00:00,2023-12-14 01:16:08+00:00,0,0,Is that jims mother in the roaster im excited to play with her
2027,rDxrpSqYHD8,@johnjohnson4834,2023-12-15 06:09:22+00:00,2023-12-15 06:09:22+00:00,1,0,how panda gets a spot is beyond me just make it a reskin of kuma ffs
2028,rDxrpSqYHD8,@noyce.,2023-12-15 12:57:04+00:00,2023-12-15 12:57:04+00:00,1,0,32 Characters off the bat LETTTTTTTTTTTTTTTS GO
2029,rDxrpSqYHD8,@ssj3134,2023-12-15 14:53:14+00:00,2023-12-15 14:53:14+00:00,1,0,no eddy
2030,rDxrpSqYHD8,@andreypastrana810,2023-12-17 05:44:57+00:00,2023-12-17 05:44:57+00:00,0,0,NEW ROSTER LOOKS SO GOOD But where is the big man himself BOB


In [21]:
def remove_digits(text):
    """Removes digits from the text.

    Parameters
    -----------
    text : str
        A string of text.

    Returns
    --------
    text : str
        The string of text without any digits.

    """
    pattern = r"\d"
    
    return re.sub(pattern, "", text)

In [22]:
df.head(20)

,videoId,authorDisplayName,publishedAt,updatedAt,likeCount,totalReplyCount,textDisplay
0,rDxrpSqYHD8,@silveriver9,2023-11-01 16:09:58+00:00,2023-11-01 16:10:43+00:00,4,4,First. Now where is LEI WULONG?!
1,rDxrpSqYHD8,@faizaanjaved7150,2023-11-01 16:10:05+00:00,2023-11-01 16:10:05+00:00,1,1,Already seen it. Ur getting less view&#39;s now bamco
2,rDxrpSqYHD8,@TS-rw4lk,2023-11-01 16:10:05+00:00,2023-11-01 16:10:05+00:00,0,0,wow
3,rDxrpSqYHD8,@ALONCAK,2023-11-01 16:10:06+00:00,2023-11-01 16:10:06+00:00,0,0,Oww yeaah
4,rDxrpSqYHD8,@Rough_Estimates,2023-11-01 16:10:06+00:00,2023-11-01 16:10:06+00:00,135,14,I hope we get an angel version of Jin
5,rDxrpSqYHD8,@kazamataurus337,2023-11-01 16:10:08+00:00,2023-11-01 16:10:08+00:00,1,0,So it begins!!!
6,rDxrpSqYHD8,@XKAMIKAZEEX,2023-11-01 16:10:10+00:00,2023-11-01 16:10:10+00:00,0,0,LET&#39;S GO
7,rDxrpSqYHD8,@bluefacebaby4167,2023-11-01 16:10:10+00:00,2023-11-01 16:10:10+00:00,1,0,Miguel waiting room right here.
8,rDxrpSqYHD8,@subzerodark,2023-11-01 16:10:11+00:00,2023-11-01 16:10:11+00:00,1,0,Wow
9,rDxrpSqYHD8,@therealwnd1820,2023-11-01 16:10:13+00:00,2023-11-01 16:10:13+00:00,0,0,Yo


In [23]:
df_no_digits = df.copy()
df_no_digits["textDisplay"] = df["textDisplay"].apply(remove_digits)
df_no_digits.head(20)

,videoId,authorDisplayName,publishedAt,updatedAt,likeCount,totalReplyCount,textDisplay
0,rDxrpSqYHD8,@silveriver9,2023-11-01 16:09:58+00:00,2023-11-01 16:10:43+00:00,4,4,First. Now where is LEI WULONG?!
1,rDxrpSqYHD8,@faizaanjaved7150,2023-11-01 16:10:05+00:00,2023-11-01 16:10:05+00:00,1,1,Already seen it. Ur getting less view&#;s now bamco
2,rDxrpSqYHD8,@TS-rw4lk,2023-11-01 16:10:05+00:00,2023-11-01 16:10:05+00:00,0,0,wow
3,rDxrpSqYHD8,@ALONCAK,2023-11-01 16:10:06+00:00,2023-11-01 16:10:06+00:00,0,0,Oww yeaah
4,rDxrpSqYHD8,@Rough_Estimates,2023-11-01 16:10:06+00:00,2023-11-01 16:10:06+00:00,135,14,I hope we get an angel version of Jin
5,rDxrpSqYHD8,@kazamataurus337,2023-11-01 16:10:08+00:00,2023-11-01 16:10:08+00:00,1,0,So it begins!!!
6,rDxrpSqYHD8,@XKAMIKAZEEX,2023-11-01 16:10:10+00:00,2023-11-01 16:10:10+00:00,0,0,LET&#;S GO
7,rDxrpSqYHD8,@bluefacebaby4167,2023-11-01 16:10:10+00:00,2023-11-01 16:10:10+00:00,1,0,Miguel waiting room right here.
8,rDxrpSqYHD8,@subzerodark,2023-11-01 16:10:11+00:00,2023-11-01 16:10:11+00:00,1,0,Wow
9,rDxrpSqYHD8,@therealwnd1820,2023-11-01 16:10:13+00:00,2023-11-01 16:10:13+00:00,0,0,Yo


# Non english comments

In [ ]:
import langdetect

def remove_words_in_languages(text, languages_to_remove):
    """Removes words from a string that are in specified languages.

    Args:
        text: The input string to process.
        languages_to_remove: A list of language codes (e.g., "fr", "es") to remove.

    Returns:
        The string with words in those languages replaced with "".
    """

    words = text.split()
    filtered_words = []
    for word in words:
        try:
            detected_language = langdetect.detect(word)
            if detected_language not in languages_to_remove:
                filtered_words.append(word)
        except langdetect.lang_detect_exception.LangDetectException:
            # Handle cases where language detection fails
            pass  # Skip the word in this case

    return " ".join(filtered_words)

# Remove 'href' 

In [24]:
def remove_href_pattern(text):
 """Removes any occurrences of "href" followed by non-whitespace characters until the next whitespace.

 Parameters
 -----------
   text: str
       The string of words to process.

 Returns
 ---------
   text : str
       The string with the "href" pattern removed.
 """
    
 pattern = r"href\S+"
    
 return re.sub(pattern, "", text)

In [25]:
sentence = "this is a hrefknljkncljknsljkcv  fwkojo"

In [26]:
sentence = remove_href_pattern(sentence)
sentence

'this is a  fwkojo'